In [ ]:
# default_exp models.mqcnn.mqcnn

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# export
import os

import numpy as np
import pandas as pd

import torch as t
import torch.nn as nn
from torch import optim

from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader_general import TimeSeriesLoader
from nixtla.models.components.common import CausalConv1d

# MQ-CNN(RNN): A Multi-Horizon Quantile Recurrent Forecaster

> API details.
- https://arxiv.org/abs/1711.11053

In [ ]:
# export
class Sencoder(nn.Module):
    def __init__(self, in_features, out_features, trnn):
        super(Sencoder, self).__init__()
        self.trnn = trnn
        layers = [nn.Dropout(p=0.5),
                  nn.Linear(in_features=in_features, out_features=out_features),
                  nn.ReLU()]
        self.encoder = nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.encoder(x)
        x = torch.cat(self.trnn * [x]) # Repeat values to match time
        return x

class Xencoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation_levels):
        super(Xencoder, self).__init__()
        self.l1_regularizer = TimeDistributed(FeaturewiseL1Regularizer(in_features=in_channels))
        self.causal_convs = CausalConvs1d(in_channels=in_channels, out_channels=out_channels, 
                                          kernel_size=kernel_size, dilation_levels=dilation_levels)
    
    def forward(self, x):
        x = self.l1_regularizer(x)
        x = self.causal_convs(x)
        return x
    
class Fencoder(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation_levels):
        super(Fencoder, self).__init__()
        self.l1_regularizer = TimeDistributed(FeaturewiseL1Regularizer(in_features=in_channels))
        self.causal_convs = CausalConvs1d(in_channels=in_channels, out_channels=out_channels, 
                                          kernel_size=kernel_size, dilation_levels=dilation_levels)
    
    def forward(self, x):
        x = self.l1_regularizer(x)
        x = self.causal_convs(x)
        return x

In [ ]:
# export
class MQCNN(object):
    """ Multi Quantile Convolutional Neural Network
    """
    def __init__(self,
                 # Architecture parameters
                 
                 # Optimization and regularization
                 
                 # Data parameters
                 input_size,
                 output_size,
                 learning_rate,
                 lr_scheduler_step_size,
                 lr_decay,
                 n_iterations,
                 early_stopping,
                 loss,
                 val_loss,
                 random_seed,
                 device=None):
        
        self.sencoder = None
        self.xencoder = None
        self.fencoder = None
        
        